In [25]:

import bibtexparser
import re

In [35]:
with open('data/BIB_saef_20241113.bib') as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

In [34]:
for entry in bib_database.entries:
    print(entry)
    # extra = entry['note']
    # names_tidy  = re.sub('saef: ', '', extra, flags=re.IGNORECASE)
    # names_split = names_tidy.strip().split(';') # Create a list of saef names
    # print(names_split)


{'file': 'Remote Sens Ecol Conserv - 2023 - Turner - Mapping water content in drying Antarctic moss communities using UAS‐borne SWIR.pdf:C\\:\\\\Users\\\\okpokam\\\\Zotero\\\\storage\\\\XVZ64ZZS\\\\Remote Sens Ecol Conserv - 2023 - Turner - Mapping water content in drying Antarctic moss communities using UAS‐borne SWIR.pdf:application/pdf', 'pages': '296--311', 'keywords': 'polar, peer reviewed, drones, hyperspectral, remote sensing, UAS, vegetation', 'note': 'saef: KrystalRandall; SharonRobinson; MelindaWaterman\nipop: UOW{\\textasciitilde}38\nproject: T2\\_P022', 'year': '2024', 'month': 'June', 'author': 'Turner, Darren and Cimoli, Emiliano and Lucieer, Arko and Haynes, Ryan S. and Randall, Krystal and Waterman, Melinda J. and Lucieer, Vanessa and Robinson, Sharon A.', 'journal': 'Remote Sensing in Ecology and Conservation', 'urldate': '2023-10-18', 'number': '3', 'language': 'en', 'abstract': 'Abstract\n\nAntarctic moss beds are sensitive to climatic conditions, and both their surv

In [14]:
print(db.get_entries())

OrderedDict()


In [13]:
recoverer = biblib.bib.messages.InputErrorRecoverer()
for ent in db.get_entries().values():
    with recoverer:
        biblib.bib.print_entry(ent)

C:\Users\okpokam\AppData\Local\Temp\ipykernel_14888\2005623139.py:1: UserWarning: InputErrorRecoverer must be reraised or disposed
  recoverer = biblib.bib.messages.InputErrorRecoverer()
